<h1 align="center">LING530F: Deep Learning for NLP</h1>
<h1 align="center">Assignment 2 : Implicit Emotion Detection</h1>
<h1 align="center">LSTM with Self Attention(Part 2) Model</h1>
<h2 align="center"> Arun Rajendran(86611860)</h2>

In [1]:
%%capture
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip install GoogleDriveDownloader
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
!pip install torchtext
!pip install spacy && python -m spacy download en

In [2]:
# Download the data

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='11surQQr3jbmzHDKgEA5c46CKA23Nf7PA',                                 # Id of file to be downloaded
                                    dest_path='data/dev.csv')                                                    # Destination path

gdd.download_file_from_google_drive(file_id='1TokPKns1uZRAW-GDN0fUz259haU2ZZAW',                                 # Id of file to be downloaded
                                    dest_path='data/test.csv')                                                    # Destination path

gdd.download_file_from_google_drive(file_id='1oBiUaUHLrXehWF570xLyQNTjD9QU4fPY',                                 # Id of file to be downloaded
                                    dest_path='data/train.csv')                                                    # Destination path

gdd.download_file_from_google_drive(file_id='1ewgcXWBioeMVJlKsZK9gLv3hWxGDqmiz',                                 # Id of file to be downloaded
                                    dest_path='data/trial-v3.labels')                                                    # Destination path

In [3]:
# Replace the fake labels in dev file with correct labels

labels = open('data/trial-v3.labels','r').readlines()
lines = open('data/dev.csv','r').readlines()
correct_lines=[]
for label, line in zip(labels, lines):
        line=line.rstrip('\n')
        text = line.split(',')
        text[0]=label
        correct_line=[''.join(text[0]),''.join(text[1:])]
        correct_lines.append(correct_line)

In [4]:
# Write the clean dev data in new file
out = open('data/clean_dev.csv',mode='w')                                      
for line in correct_lines:
    line[0]=line[0].rstrip('\n')
    out.write(line[0])
    out.write(',')
    out.write(line[1])
    out.write('\n')
out.close()     

In [5]:
# Read train and dev data
%%capture  # As training data has some error lines
import pandas as pd
X_train = pd.read_table('data/train.csv',sep=',',index_col=False, error_bad_lines=False)
X_test = pd.read_csv('data/clean_dev.csv',sep=',')

In [6]:
# Pandas read skips the error lines, then we save the new dataset into separate file

X_train.to_csv('data/clean_train.csv',sep=',',index=False)

In [7]:
y_train=X_train.label
X_train = X_train.drop(['label'],axis=1)
y_test=X_test.label1
X_test = X_test.drop(['label1'],axis=1)

In [8]:
import torch 
import torch.nn as nn
import torchvision.datasets as datasets
from skimage import transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from collections import Counter,OrderedDict
from keras.utils.vis_utils import *
import random
import math

Using TensorFlow backend.


In [9]:
# Define the vector of glove embeddings

%%capture  # Not display output
from torchtext import vocab
# specify the path to the localy saved vectors
vec = vocab.GloVe(name='840B', dim=300)
# vec = vocab.GloVe(name='6B', dim=200)

In [10]:
import torchtext
import spacy
import re
from torchtext import data

# tokenizer function using spacy
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
def tokenizer(s): 
    return [w.text.lower() for w in nlp(tweet_clean(s))]

def tweet_clean(text):
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text) # remove non alphanumeric character
    text = re.sub(r'https?:/\/\S+', ' ', text) # remove links
    return text.strip()

def tokenizer(s): 
    return [w.text.lower() for w in nlp(tweet_clean(s))]

# define the columns that we want to process and how to process
TEXT = data.Field(sequential=True, 
                 tokenize=tokenizer, 
                 include_lengths=True, 
                 use_vocab=True)
LABEL = data.Field(sequential=False, 
                   use_vocab=True,
                   pad_token=None, 
                   unk_token=None)

train_val_fields = [
    ('label', LABEL), # process it as label
    ('tweet', TEXT) # process it as text
]

trainds, valds = data.TabularDataset.splits(path='data', 
                                            format='csv', 
                                            train='clean_train.csv', 
                                            validation='clean_dev.csv', 
                                            fields=train_val_fields, 
                                            skip_header=True)

In [11]:
# build the vocabulary using train and validation dataset and assign the vectors
TEXT.build_vocab(trainds,valds, max_size=100000, vectors=vec)
# build vocab for labels
LABEL.build_vocab(trainds)

In [12]:
print(TEXT.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.6339,  0.1782, -0.5805,  ...,  0.3635, -0.3022, -0.0209],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.8860, -0.0697, -0.3037,  ...,  0.4366, -0.3720, -0.0778]])


In [13]:
print(LABEL.vocab.vectors)

None


In [14]:
# Create bucketiterator to get batches

traindl, valdl = data.BucketIterator.splits(datasets=(trainds, valds), # specify train and validation Tabulardataset
                                            batch_sizes=(64,64),  # batch size of train and validation
                                            sort_key=lambda x: len(x.tweet), # on what attribute the text should be sorted
                                            device=None, # -1 mean cpu and 0 or None mean gpu
                                            sort_within_batch=True, 
                                            repeat=False)

In [15]:
print(len(traindl), len(valdl))

2365 150


In [16]:
traindl

In [17]:
batch = next(iter(traindl)) # BucketIterator return a batch object

In [18]:
print(batch.label) # labels of the batch

tensor([1, 0, 4, 0, 2, 1, 0, 4, 5, 2, 1, 3, 0, 0, 2, 0, 0, 3, 3, 0, 1, 1, 5, 4,
        1, 0, 1, 5, 2, 2, 4, 1, 1, 0, 3, 4, 0, 2, 0, 4, 0, 1, 5, 5, 4, 4, 4, 0,
        1, 0, 3, 4, 3, 1, 0, 2, 3, 3, 0, 3, 0, 0, 0, 0])


In [19]:
print(batch.tweet) 

(tensor([[  285,  1102,     5,     5,    68,    10,     5,     3,     5,     5,
             5,   283,    81,     5,     5,     2,     5,    14,   326,     3,
           106,     3,     3,     2,     5,     3,     3,    36,     5,    45,
             3,    13,     3,   920,     7,    30,    10,     5,     3,     3,
            14,    16,     3,     5,     3,   298,     3,     5,   104,  4089,
             3,     3,     3,     5,    14,   205,     7,     9, 53238,     3,
             5, 10889,     5,     5],
        [  243,     8,     3, 12453,     2,    49,     5,    31,     3,   106,
             3,   146,    25,    44,     2,     7,    10,     2,     3,    31,
             2,    79,    11,    17,    26,    79,   117,     2,   176,    18,
            11,     9,    79,     2,    26,    15,    97,     3,    11,    21,
           145,     9,    79,     9,    21,     2,    11,    48,     2,    34,
            31,    11,    21,    33,     2,    89,    51,    60,    40,    31,
           32

In [20]:
print(batch.dataset.fields)

{'label': <torchtext.data.field.Field object at 0x7efd473a5588>, 'tweet': <torchtext.data.field.Field object at 0x7efcc4df7080>}


In [21]:
# Create batchgenerator class to use Bucketiterator and attributes to get the batches

class BatchGenerator:
    def __init__(self, dl, x_field, y_field):
        self.dl, self.x_field, self.y_field = dl, x_field, y_field
        
    def __iter__(self):
        for batch in self.dl:
            X = getattr(batch, self.x_field)
            y = getattr(batch, self.y_field)
            yield (X,y)
 


In [22]:
train_batch_it = BatchGenerator(traindl, 'tweet', 'label')
valid_batch_it = BatchGenerator(valdl, 'tweet', 'label')

In [23]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
 
# Define the class for LSTM model with attention         
    
class AttentionLSTM(nn.Module):
    def __init__(self, hidden_dim, n_lstm_layers,emb_dim, dropout_p):
        super(AttentionLSTM,self).__init__() # don't forget to call this!
        self.embedding = nn.Embedding(len(TEXT.vocab), emb_dim)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors, requires_grad=False)
        self.encoder = nn.LSTM(emb_dim, hidden_dim, num_layers=n_lstm_layers)
        self.dropout = nn.Dropout(dropout_p)
        self.fc1=nn.Linear(hidden_dim,128)
        self.predictor=nn.Linear(128,6)
 
    def attention_net(self, lstm_output, final_state):
        hidden = final_state.squeeze(0)
        attn_weights = torch.bmm(lstm_output, hidden.unsqueeze(2)).squeeze(2)
        soft_attn_weights = F.softmax(attn_weights, 1)
        new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        return new_hidden_state
  
    def forward(self, seq):
        output, (final_hidden_state, final_cell_state) = self.encoder(self.embedding(seq))
        output = output.permute(1, 0, 2) # output.size() = (batch_size, num_seq, hidden_size)
        attn_output = self.attention_net(output, final_hidden_state)
        attn_output= self.dropout(attn_output)
        fc_output= self.fc1(attn_output)
        fc_output= self.dropout(fc_output)
        preds=self.predictor(fc_output)
        return preds

In [24]:
embed_size = 300
nh = 256
drop=0.5
n_lstm_layers=1
model = AttentionLSTM(nh,n_lstm_layers,embed_size,drop)

In [25]:
from tqdm import tqdm
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import StepLR
opt = optim.Adam(model.parameters(), lr=1e-4)
loss_func = nn.CrossEntropyLoss()
scheduler = StepLR(opt, step_size=1, gamma=0.5) 
epochs = 4
batch_size = 64 
val_list_fscores=[]

state = torch.load('mytraining1.pt')
model.load_state_dict(state)
for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    iterations=0
    scheduler.step()
    model.train() # turn on training mode
    for x,y in tqdm(train_batch_it,disable=True): # thanks to our wrapper, we can intuitively iterate over our data!
        iterations=iterations+1
        opt.zero_grad()
        (x,lengths)=x
        predicted = model(x)
        y=y.view(-1)
        loss = loss_func(predicted,y)
        loss.backward()
        opt.step()
        running_loss += loss.item() * x.size(0)
    epoch_loss = running_loss / len(trainds)
 
    # calculate the validation loss for this epoch
    val_loss = 0.0
    list_scores=[]
    model.eval() # turn on evaluation mode
    for x,y in tqdm(valid_batch_it,disable=True):
        (x,lengths)=x
        preds = model(x)
        y=y.view(-1)
        loss = loss_func(preds,y)
        val_loss += loss.item() * x.size(0)
        _,y_pred=torch.max(preds,1)
        temp=f1_score(y_pred,y, average='macro', labels=np.unique(y_pred))
        list_scores.append(temp)
    
    val_loss /= len(valds)
    fscore = sum(list_scores)/(len(valds)/batch_size)
    val_list_fscores.append(fscore)
    torch.save(model.state_dict(), 'mytraining'+str(epoch)+'.pt')
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, F1 Sum:{:.4f},Average F1 score(Validation Set): {:.4f}'.format(epoch, epoch_loss, val_loss,sum(list_scores),fscore))


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch: 1, Training Loss: 0.2110, Validation Loss: 0.3362, F1 Sum:93.0315,Average F1 score(Validation Set): 0.6211
Epoch: 2, Training Loss: 0.2002, Validation Loss: 0.3428, F1 Sum:93.3294,Average F1 score(Validation Set): 0.6231
Epoch: 3, Training Loss: 0.1953, Validation Loss: 0.3487, F1 Sum:93.0176,Average F1 score(Validation Set): 0.6210
Epoch: 4, Training Loss: 0.1938, Validation Loss: 0.3511, F1 Sum:93.1964,Average F1 score(Validation Set): 0.6222


In [26]:
# Save the model with highest F score

index = np.argmax(val_list_fscores)
from google.colab import files
files.download('mytraining'+str(index+1)+'.pt')

### References

NLTK: Stemming and Lemmatization<br\>
https://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization

Apply Stemmer to a column<br\>
https://stackoverflow.com/questions/43795310/apply-porters-stemmer-to-a-pandas-column-for-each-word

Ensemble: Voting Classifier<br\>
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

Sample Pipeline<br\>
http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html

TorchText and LSTM<br\>
http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

TorchText and CNN<br\>
https://github.com/DSKSD/DeepNLP-models-Pytorch/blob/master/notebooks/08.CNN-for-Text-Classification.ipynb
https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

Saving and loading Pytorch Model<br\>
https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch

Self Attention Explained<br\>
https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#self-attention

Self Attention Code for LSTM<br\>
https://github.com/prakashpandey9/Text-Classification-Pytorch/blob/master/models/LSTM_Attn.py

LSTM + CNN model 
https://discuss.pytorch.org/t/cnn-layer-in-the-top-of-lstm/7941/5
https://discuss.pytorch.org/t/solved-concatenate-time-distributed-cnn-with-lstm/15435/4